In [13]:
parsed_file = "test_file"
word_occurence_map = {}

with open(parsed_file) as inp:
    prev_line, prev_count = '', 0
    for line in inp:
        curr_line = "\t".join(line.strip().split("\t")[:-1])
        if curr_line == prev_line or (not prev_line):
            prev_count += int(line.strip().split("\t")[-1])
        else:
            print (curr_line)
            x, y, path = prev_line.split('\t')
            key = str(x) + '_' + str(y)
            current = path + ":" + str(prev_count)
            if key in word_occurence_map:
                pastkeys = word_occurence_map[key]
                current =  pastkeys + ',' + current
            word_occurence_map[key] = current
            prev_count = 1
        prev_line = curr_line
    x, y, path = prev_line.split('\t')
    key = str(x) + '_' + str(y)
    current = path + ":" + str(prev_count)
    if key in word_occurence_map:
        pastkeys = word_occurence_map[key]
        current =  pastkeys + ',' + current
    word_occurence_map[key] = current
    


19193456	23452389	9668810
663726	9048846	4107215
6178566	6097931	8733868
8184873	13389052	6536081
5933740	15640897	8593453
1837912	2946514	9443662
16864707	22008180	11071817
13583494	15294065	657327
18679314	8042043	11954734
18679314	8042040	11954736
18679314	8042043	11954739


In [23]:
import sys, shelve, picklw
file = "damru"
p = pickle.load(open(file, 'rb'))
db_file = ".".join(file.split(".")[:-1]) + ".db" 
with shelve.open(db_file, 'c') as db:
    for k in p:
        try:
            p[k] = p[k].decode("utf-8")
        except:
            pass
        try:
            k = k.decode("utf-8")
        except:
            pass

        if k and p[k]:
            db[str(k)] = str(p[k])
print ("db_file", db_file, "done")

KeyboardInterrupt: 

In [1]:
completed_files = """db_file security_threshold_3_8/security_relations_map.db done
db_file security_threshold_3_10/security_relations_map.db done
db_file security_threshold_3_4/security_relations_map.db done
db_file security_threshold_3_6/security_relations_map.db done
db_file security_threshold_3_15/security_relations_map.db done
db_file security_threshold_7_10/security_relations_map.db done
db_file security_threshold_7_30/security_relations_map.db done
db_file security_threshold_7_4/security_relations_map.db done
db_file security_threshold_7_8/security_relations_map.db done
db_file security_threshold_7_6/security_relations_map.db done
db_file security_threshold_7_15/security_relations_map.db done
db_file security_threshold_5_8/security_relations_map.db done
db_file security_threshold_5_6/security_relations_map.db done"""

all_files = """-rw-rw-r-- 1 vlead vlead 7191658496 Apr 16 06:00 security_threshold_10_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2106208256 Apr 16 15:37 security_threshold_10_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1817264128 Apr 16 15:37 security_threshold_10_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2585493504 Apr 16 15:37 security_threshold_10_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1807597568 Apr 16 15:37 security_threshold_10_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2572136448 Apr 16 15:37 security_threshold_10_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1650249728 Apr 16 15:37 security_threshold_12_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2968449024 Apr 16 15:37 security_threshold_12_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1815060480 Apr 16 15:37 security_threshold_12_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1776738304 Apr 16 15:37 security_threshold_12_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2283626496 Apr 16 15:37 security_threshold_12_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2369228800 Apr 16 15:37 security_threshold_12_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1719312384 Apr 16 15:37 security_threshold_15_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2717384704 Apr 16 15:37 security_threshold_15_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2779189248 Apr 16 15:37 security_threshold_15_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1787883520 Apr 16 15:37 security_threshold_15_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2766249984 Apr 16 15:37 security_threshold_15_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2936958976 Apr 16 15:37 security_threshold_15_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1858682880 Apr 16 15:37 security_threshold_20_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2116255744 Apr 16 15:37 security_threshold_20_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1622700032 Apr 16 15:37 security_threshold_20_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1764642816 Apr 16 15:37 security_threshold_20_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1969885184 Apr 16 15:37 security_threshold_20_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2415542272 Apr 16 15:37 security_threshold_20_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1578115072 Apr 16 15:37 security_threshold_25_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1587240960 Apr 16 15:37 security_threshold_25_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1588240384 Apr 16 15:37 security_threshold_25_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1571401728 Apr 16 15:37 security_threshold_25_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1769127936 Apr 16 15:37 security_threshold_25_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2425806848 Apr 16 15:37 security_threshold_25_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead      12288 Apr 16 06:59 security_threshold_3_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead      12288 Apr 16 06:59 security_threshold_3_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead      12288 Apr 16 06:59 security_threshold_3_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead      12288 Apr 16 06:59 security_threshold_3_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead      12288 Apr 16 06:59 security_threshold_3_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 1869680640 Apr 16 15:37 security_threshold_4_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 5675888640 Apr 16 15:37 security_threshold_4_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 5366837248 Apr 16 15:37 security_threshold_4_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 5314387968 Apr 16 15:37 security_threshold_4_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 6918279168 Apr 16 15:37 security_threshold_4_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 3676905472 Apr 16 15:37 security_threshold_4_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 3756224512 Apr 16 15:37 security_threshold_50_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 5083078656 Apr 16 15:37 security_threshold_50_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 4478525440 Apr 16 15:37 security_threshold_50_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 5521424384 Apr 16 15:37 security_threshold_50_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 3211071488 Apr 16 15:37 security_threshold_50_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2976628736 Apr 16 15:37 security_threshold_50_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 2801909760 Apr 16 15:37 security_threshold_5_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7253737472 Apr 16 15:37 security_threshold_5_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7273426944 Apr 16 15:37 security_threshold_5_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7275110400 Apr 16 15:37 security_threshold_5_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7398264832 Apr 16 13:25 security_threshold_5_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7401832448 Apr 16 12:53 security_threshold_5_8/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7262851072 Apr 16 09:33 security_threshold_7_10/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7259885568 Apr 16 12:16 security_threshold_7_15/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7258484736 Apr 16 10:22 security_threshold_7_30/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7247130624 Apr 16 10:28 security_threshold_7_4/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7257264128 Apr 16 11:37 security_threshold_7_6/security_relations_map.db
-rw-rw-r-- 1 vlead vlead 7260504064 Apr 16 11:17 security_threshold_7_8/security_relations_map.db"""

completed_files = [l.split()[1] for l in completed_files.split("\n")] 
all_files = [l.split()[-1] for l in all_files.split("\n")] 
incomplete_files = [file for file in all_files if file not in completed_files]

In [5]:
for f in incomplete_files:
    !rm ../junk/Files/$f

In [9]:
dataset = open("../files/dataset/dataset.tsv").read().split("\n")
dataset_t = [l for l in dataset if l.split("\t")[-1]!="none"]
dataset_f = [l for l in dataset if l.split("\t")[-1]=="none"]
import random
dataset_t_mini = random.choices(dataset_t, k=int(0.1 * len(dataset_t)))
dataset_f_mini = random.choices(dataset_f, k=10000-int(0.1 * len(dataset_t)))
dataset_mini = dataset_t_mini + dataset_f_mini
random.shuffle(dataset_mini)
open("../files/dataset/dataset_mini.tsv","w+").write("\n".join(dataset_mini))

336853

In [6]:
len(dataset_f_mini)

9240